In [ ]:
import pickle


In [ ]:
#loading the train_copus_padded data from .pickle file
file = open('pickle_data/train_copus_pad.pickle','rb')
train_copus_padded = pickle.load(file)

file = open('pickle_data/test_copus_pad.pickle','rb')
test_copus_padded = pickle.load(file)

file = open('pickle_data/vocab_train.pickle','rb')
vocab_to_int_train = pickle.load(file)

file = open('pickle_data/embedding_matrix','rb')
embedding_matrix = pickle.load(file)
file.close()

#the size of vocabulary
copus_size = len(vocab_to_int_train)
#the maximal length of every sentence
maxlen = len(train_copus_padded[0][0])
#dimension of embedding matrix
dimension = 300

In [ ]:
#*******start to embedding the train_data

#one by one sentence of each review as input
#each_review is 36*224
train_copus_padded = train_copus_padded[:5]
embedding_layer = Embedding(copus_size,dimension,weights = [embedding_matrix],input_length = maxlen,trainable = False)
for review in train_copus_padded:
    for sent in review:
        sent = tf.convert_to_tensor(sent)
        sent = tf.reshape(sent,[1,224])
        embeded_sequence = embedding_layer(sent)
        #合并每个vector
        embeded_sequences = tf.concat(embeded_sequence,0)

In [ ]:
print(embeded_sequences)